<h1 align=center> Facial Expression Recognition Classifier Model</h1>



### Import Libraries
- Let us import all the required Libraries and Packages including Tensorflow


In [1]:
import os
import utils
import numpy as np
import seaborn as sns
import tensorflow as tf
tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

import matplotlib.pyplot as plt

# To set the backend of matplotlib to the 'inline' backend
%matplotlib inline

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D


1 Physical GPUs, 1 Logical GPUs


### Plot Sample Images
- Let us have a look at random images from the Dataset

In [2]:
utils.datasets.fer.plot_example_images(plt).show()

AttributeError: module 'utils' has no attribute 'datasets'

### Data Sets
- Let us look at the number of Images in Training and Testing Datasets

In [3]:

print("Images in Training Data")
print("_______________________")
for expression in os.listdir("training_data/"):
    print(str(len(os.listdir("training_data/" + expression))) + " " + expression + " images")

print("\n")

print("Images in Testing Data")
print("_______________________")
for expression in os.listdir("testing_data/"):
    print(str(len(os.listdir("testing_data/" + expression))) + " " + expression + " images")
    


Images in Training Data
_______________________
3995 angry images
436 disgust images
4097 fear images
7215 happy images
4965 neutral images
4830 sad images
3171 surprise images


Images in Testing Data
_______________________
958 angry images
111 disgust images
1024 fear images
1774 happy images
1233 neutral images
1247 sad images
831 surprise images


### Create Training and Validation Batches
- Using the ImageDataGenerators Let us create Training and Validation Batches by loading images from corresponding directories

In [4]:
img_size = 48
batch_size = 64

datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("training_data/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

datagen_validation = ImageDataGenerator(horizontal_flip=True)
validation_generator = datagen_validation.flow_from_directory("testing_data/",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


### Create Convolutional Nueral Network (CNN) Model
- Let us create a Nueral Network using 4 Convolutional Layers and 2 Fully Connected dense Layers.

In [5]:
# Initialising the CNN
model = Sequential()

# 1st Convolution Layer

# There are 64 (3,3) filters with "same" Padding and Shape of the Input_Image is (48,48,1)
model.add(Conv2D(64,(3,3), padding='same', input_shape=(48, 48,1)))

# Normalizing to speed up learning.
model.add(BatchNormalization())

# Applying Non Linear Activation Function "relu"
model.add(Activation('relu'))  

# Adding a Max Pool Layer of size (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with 0.25 fraction of the input units to drop
model.add(Dropout(0.25))




# 2nd Convolution layer

# There are 128 (5,5) filters with "same" Padding 
model.add(Conv2D(128,(5,5), padding='same'))

# Normalizing to speed up learning.
model.add(BatchNormalization())

# Applying Non Linear Activation Function "relu"
model.add(Activation('relu'))

# Adding a Max Pool Layer of size (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with 0.25 fraction of the input units to drop
model.add(Dropout(0.25))




# 3rd Convolution layer

# There are 512 (3,3) filters with "same" Padding 

model.add(Conv2D(512,(3,3), padding='same'))

# Normalizing to speed up learning.
model.add(BatchNormalization())

# Applying Non Linear Activation Function "relu"
model.add(Activation('relu'))

# Adding a Max Pool Layer of size (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with 0.25 fraction of the input units to drop
model.add(Dropout(0.25))





# 4th Convolution layer

# There are 512 (3,3) filters with "same" Padding 
model.add(Conv2D(512,(3,3), padding='same'))

# Normalizing to speed up learning.
model.add(BatchNormalization())

# Applying Non Linear Activation Function "relu"
model.add(Activation('relu'))

# Adding a Max Pool Layer of size (2,2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropout layer with 0.25 fraction of the input units to drop 
model.add(Dropout(0.25))



# Flattening
model.add(Flatten())


# Fully connected layer with 256 nuerons
model.add(Dense(256))

# Normalizing to speed up learning.
model.add(BatchNormalization())

# Applying Non Linear Activation Function "relu"
model.add(Activation('relu'))

# Dropout layer with 0.25 fraction of the input units to drop
model.add(Dropout(0.25))



# Fully connected layer with 512 nuerons
model.add(Dense(512))

# Normalizing to speed up learning.
model.add(BatchNormalization())

# Applying Non Linear Activation Function "relu"
model.add(Activation('relu'))

# Dropout layer with 0.25 fraction of the input units to drop
model.add(Dropout(0.25))

# Adding a final Dense Layer with 7 outputs corresponding to 7 different emotions with a "softmax" Activation Function 
model.add(Dense(7, activation='softmax'))

### Compiling the Model
- Let us use Adam Optimizer

In [6]:
# Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent 
# procedure to update network weights iterative based in training data.

# Let us choose a Learning rate of 0.0005 
opt = Adam(lr=0.0005)


# Compile defines the loss function, the optimizer and the metrics.

# As we have Categorical Values we will use 'categorical_crossentropy' loss function
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Let us check the details of the Model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 128)       5

### Train and Evaluate Model

In [7]:
# Let us train the Model 15 times
epochs = 15

steps_per_epoch = train_generator.n//train_generator.batch_size
validation_steps = validation_generator.n//validation_generator.batch_size

# Create a Callback which reduces the Learning rate by a factor of "0.1" when the val_loss does not decrease
# after 2 epochs also and allowing the minimum value of Learning Rate to be 0.00001
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')


# Create another Callback which saves the Model Weights by monitoring the Val_Accuracy
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)


# A callback is an object that can perform actions at various stages of training
# (e.g. at the start or end of an epoch, before or after a single batch, etc).
callbacks = [checkpoint, reduce_lr]

# Fitting the model .
history = model.fit(
    x=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = validation_generator,
    validation_steps = validation_steps,
    callbacks=callbacks
)

Epoch 1/15
448/448 [==============================] - 125s 266ms/step - loss: 1.9487 - accuracy: 0.2566 - val_loss: 1.7709 - val_accuracy: 0.3619

Epoch 00001: saving model to model_weights.h5
Epoch 2/15
448/448 [==============================] - 11s 23ms/step - loss: 1.5274 - accuracy: 0.4136 - val_loss: 1.3784 - val_accuracy: 0.4694

Epoch 00002: saving model to model_weights.h5
Epoch 3/15
448/448 [==============================] - 11s 24ms/step - loss: 1.3467 - accuracy: 0.4879 - val_loss: 1.2862 - val_accuracy: 0.4960

Epoch 00003: saving model to model_weights.h5
Epoch 4/15
448/448 [==============================] - 11s 24ms/step - loss: 1.2591 - accuracy: 0.5184 - val_loss: 1.1928 - val_accuracy: 0.5442

Epoch 00004: saving model to model_weights.h5
Epoch 5/15
448/448 [==============================] - 11s 23ms/step - loss: 1.1938 - accuracy: 0.5464 - val_loss: 1.1813 - val_accuracy: 0.5536

Epoch 00005: saving model to model_weights.h5
Epoch 6/15
448/448 [=======================

### Represent Model as JSON String

In [8]:
# Converting the model into JSON format and storing it in "fer_model.json" file. 
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)